In [53]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [54]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,
    sin_generator,    
    mape
)
from gru_with_adam import GRUAdam
import plotly.graph_objects as go
from math import pi

In [55]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence)        
    # scaled = sequence  
    X, y = create_sliding_window_batches(scaled, window_size, batch_size, n)
    if not use_adam:
        model = GRUModel(window_size, hidden_size, n)
    else:
        model = GRUAdam(window_size, hidden_size, n)
    model.train(X, y, lr, max_epochs, verbosity=verbosity)
    train_pred, _, _ = model.forward(X)
    training_mape = mape(y, train_pred)           
    x_last = np.vstack(tuple(scaled[-window_size:] for _ in range(batch_size)))
    x_last = np.expand_dims(x_last, axis=0)    
    pred, _, _ = model.forward(x_last)
    descaled_pred = descale_sequence(pred[-1][-1], mean, deviation)
    # descaled_pred = pred[-1][-1]
    return training_mape, model, descaled_pred, mean, deviation

In [56]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 6
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.183219
MAPE: 81.713292%
TRAINING FINISHED
Epoch 100/100, Loss: 0.245824
MAPE: 97.319597%
TRAINING FINISHED
Epoch 100/100, Loss: 0.213274
MAPE: 97.276740%
TRAINING FINISHED
Epoch 100/100, Loss: 0.215889
MAPE: 104.716029%
TRAINING FINISHED
Epoch 100/100, Loss: 0.172576
MAPE: 83.801975%
TRAINING FINISHED
Epoch 200/200, Loss: 0.086650
MAPE: 69.491097%
TRAINING FINISHED
Epoch 200/200, Loss: 0.154460
MAPE: 86.888960%
TRAINING FINISHED
Epoch 200/200, Loss: 0.097917
MAPE: 68.467048%
TRAINING FINISHED
Epoch 200/200, Loss: 0.128618
MAPE: 74.005516%
TRAINING FINISHED
Epoch 200/200, Loss: 0.136882
MAPE: 83.319917%
TRAINING FINISHED
Epoch 300/300, Loss: 0.053142
MAPE: 48.297127%
TRAINING FINISHED
Epoch 300/300, Loss: 0.050055
MAPE: 46.865870%
TRAINING FINISHED
Epoch 300/300, Loss: 0.069713
MAPE: 59.921694%
TRAINING FINISHED
Epoch 300/300, Loss: 0.074936
MAPE: 70.825347%
TRAINING FINISHED
Epoch 300/300, Loss: 0.069255
MAPE: 66.564396%
TRAINING FINISHED
Epoch 

In [57]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [58]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 1
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 500
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 500/500, Loss: 0.122198
MAPE: 61.792311%
TRAINING FINISHED
Epoch 500/500, Loss: 0.256110
MAPE: 102.774528%
TRAINING FINISHED
Epoch 500/500, Loss: 0.250605
MAPE: 100.656882%
TRAINING FINISHED
Epoch 500/500, Loss: 0.230501
MAPE: 94.894570%
TRAINING FINISHED
Epoch 500/500, Loss: 0.108553
MAPE: 64.501710%
TRAINING FINISHED
Epoch 500/500, Loss: 0.203620
MAPE: 90.663798%
TRAINING FINISHED
Epoch 500/500, Loss: 0.257336
MAPE: 104.838779%
TRAINING FINISHED
Epoch 500/500, Loss: 0.082164
MAPE: 56.983724%
TRAINING FINISHED
Epoch 500/500, Loss: 0.069825
MAPE: 44.445954%
TRAINING FINISHED
Epoch 500/500, Loss: 0.051090
MAPE: 39.607612%
TRAINING FINISHED
Epoch 500/500, Loss: 0.139117
MAPE: 68.282225%
TRAINING FINISHED
Epoch 500/500, Loss: 0.103826
MAPE: 64.989631%
TRAINING FINISHED
Epoch 500/500, Loss: 0.080946
MAPE: 72.485033%
TRAINING FINISHED
Epoch 500/500, Loss: 0.144690
MAPE: 68.997536%
TRAINING FINISHED
Epoch 500/500, Loss: 0.146871
MAPE: 69.434875%
TRAINING FINISHED
Epoc

In [59]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [60]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 40
window_sizes = list(range(2, seq_length // 2, 2))
hidden_size = 8
batch_size = 1
output_size = 3

verbosity = 100000
lr = 1e-2
max_epochs = 250
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in window_sizes:        

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)              
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 250/250, Loss: 0.048720
MAPE: 48.719204%
TRAINING FINISHED
Epoch 250/250, Loss: 0.035868
MAPE: 49.571113%
TRAINING FINISHED
Epoch 250/250, Loss: 0.091457
MAPE: 48.266458%
TRAINING FINISHED
Epoch 250/250, Loss: 0.048372
MAPE: 41.541267%
TRAINING FINISHED
Epoch 250/250, Loss: 0.111439
MAPE: 53.291036%
TRAINING FINISHED
Epoch 250/250, Loss: 0.022475
MAPE: 48.231422%
TRAINING FINISHED
Epoch 250/250, Loss: 0.036679
MAPE: 58.509946%
TRAINING FINISHED
Epoch 250/250, Loss: 0.052383
MAPE: 58.493765%
TRAINING FINISHED
Epoch 250/250, Loss: 0.037505
MAPE: 63.483636%
TRAINING FINISHED
Epoch 250/250, Loss: 0.058443
MAPE: 48.309907%
TRAINING FINISHED
Epoch 250/250, Loss: 0.029365
MAPE: 62.204312%
TRAINING FINISHED
Epoch 250/250, Loss: 0.023148
MAPE: 65.377842%
TRAINING FINISHED
Epoch 250/250, Loss: 0.035619
MAPE: 76.174040%
TRAINING FINISHED
Epoch 250/250, Loss: 0.030799
MAPE: 72.565365%
TRAINING FINISHED
Epoch 250/250, Loss: 0.040866
MAPE: 78.310398%
TRAINING FINISHED
Epoch 2

In [61]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [62]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 10
window_size = 3
batch_size = 1
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.069315
MAPE: 106.606362%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.037164
MAPE: 73.649958%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.103106
MAPE: 118.619316%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.043945
MAPE: 81.270103%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.088307
MAPE: 124.658718%
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.006152
MAPE: 28.845831%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005922
MAPE: 28.491753%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001616
MAPE: 11.799408%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.012792
MAPE: 42.713602%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.002754
MAPE: 19.437776%
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005417
MAPE: 23.355729%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001966
MAPE: 15.491020%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.00125

In [63]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()

In [64]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.088438
MAPE: 2.888482%
TEST
Предсказания на тестовой выборке:
 [17.00577661 18.37214946 19.55869165]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 3.6782%


In [65]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 9.891472
MAPE: 4.988450%
TEST
Предсказания на тестовой выборке:
 [142.7695358  230.84261034 373.49654181]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 0.9032%


In [66]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.000935
MAPE: 33.357387%
TEST
Предсказания на тестовой выборке:
 [0.05546381 0.05277901 0.04147735]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 5.8302%


In [67]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 2000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

Epoch 2000/2000, Loss: 0.300272
MAPE: 76.522467%
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.300165
MAPE: 76.508329%
TEST
Предсказания на тестовой выборке:
 [-0.1763651   0.19311535 -0.4053049 ]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 74.1738%


In [68]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 1
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.029547
MAPE: 0.582809%
TEST
Предсказания на тестовой выборке:
 [63.92575511 81.14614529 99.84609925]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.1501%
